In [ ]:
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import cv2
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import DataLoader

WORK_DIR = Path(Path.cwd()).parent
sys.path.append(str(WORK_DIR))
from src.datasets import get_dataset, get_dataloader
from src.utils import parse_data_cfg, IMG, DATA_DIR, RHD

In [ ]:
cfg = parse_data_cfg(WORK_DIR/'data_cfg'/'znb_lift_data_mod1.cfg')
epoch = 200
exp_dir = cfg["exp_dir"]
data_split = 'test'
split_set = cfg[data_split + '_set']
img_size = int(cfg['img_size'])

# Visualize Dataloader

In [ ]:
dataset_kwargs = {'split_set': split_set}
cfg['visual'] = True # for visualisation
dataset   = get_dataset(cfg, dataset_kwargs)
sampler   = None
shuffle   = cfg['shuffle']
kwargs = {'batch_size'  :   int(cfg['batch_size']),
          'shuffle'     :   shuffle,
          'num_workers' :   int(cfg['num_workers']),
          'pin_memory'  :   True}
data_loader = get_dataloader(dataset, sampler, kwargs)


In [ ]:
idx = 0
for i, (scoremap, xyz_gt_canon, rot_mat, uv_gt, kpt_scale, crop, K, xyz_gt, img, hand_side) in enumerate(data_loader):
    if i == idx:
        img = img.cpu().numpy()
        img = IMG.scale_img_255(img)
        crop = crop.cpu().numpy()
        crop = IMG.scale_img_255(crop)        
        scoremap = scoremap.cpu().numpy()
        scoremap = np.swapaxes(scoremap, 2, 3)
        scoremap = np.swapaxes(scoremap, 1, 3)
        xyz_gt_canon = xyz_gt_canon.cpu().numpy()
        rot_mat = rot_mat.cpu().numpy()
        uv_gt = uv_gt.cpu().numpy()
        kpt_scale = kpt_scale.cpu().numpy()
        xyz_gt = xyz_gt.cpu().numpy()
        hand_side = hand_side.numpy()
        break
    i += 1

In [ ]:
idx = 0
cur_smap       = scoremap[idx]
cur_img        = img[idx]
cur_crop       = crop[idx]
cur_uv         = uv_gt[idx]
cur_rot_mat    = rot_mat[idx]
cur_xyz_canon  = xyz_gt_canon[idx]
cur_kpt        = kpt_scale[idx]
cur_K          = K[idx]
cur_xyz        = xyz_gt[idx]
cur_hand_side  = hand_side[idx]

fig, ax = plt.subplots(5, 5, figsize=(10, 10))
for i in range(5):
    for j in range(5):
        k = np.ravel_multi_index((i, j), (5, 5))
        if k >= cur_smap.shape[2]:
            break        
        show_scoremap = cv2.cvtColor(cur_smap[:, :, k], cv2.COLOR_GRAY2RGB)
        show_scoremap = cv2.normalize(show_scoremap, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC3)
        show_scoremap[:, :, 0] = 0
        show_scoremap[:, :, 2] = 0
        show = 0.5*show_scoremap + 0.5*cur_crop
    
        ax[i, j].imshow(show.astype('uint32'))
        ax[i, j].plot(cur_uv[k, 0], cur_uv[k, 1], 'b.')
        RHD.visualize_joints_2d(ax[i, j], cur_uv[RHD.REORDER_IDX])

fig, ax = plt.subplots()
ax.imshow(cur_crop)
RHD.visualize_joints_2d(ax, cur_uv[RHD.REORDER_IDX])

In [ ]:
fig, ax = plt.subplots(1, 2)
RHD.visualize_joints_2d(ax[0], cur_xyz_canon[RHD.REORDER_IDX])
RHD.visualize_joints_2d(ax[1], np.matmul(cur_xyz_canon, cur_rot_mat)[RHD.REORDER_IDX])

if cur_hand_side == 1:
    print('swapping hand sides')
    cur_xyz_canon[:, 2] = -cur_xyz_canon[:, 2]
    
reform_xyz = np.matmul(cur_xyz_canon, cur_rot_mat)*cur_kpt
reform_xyz += cur_xyz[0, :]
print(np.allclose(reform_xyz, cur_xyz))
reform_uvd = RHD.xyz2uvd(reform_xyz, cur_K)
print(np.allclose(reform_uvd, RHD.xyz2uvd(cur_xyz, cur_K)))
fig, ax = plt.subplots()
ax.imshow(cur_img)
RHD.visualize_joints_2d(ax, reform_uvd[RHD.REORDER_IDX])

# Evaluation

In [ ]:
xyz_gt_list, uv_gt_list, vis_list, K_list, hand_side_list = RHD.get_all_anno(split_set)

gt_canon_list = []
rot_mat_list = []
kpt_scale_list = []
for i, xyz in enumerate(xyz_gt_list):
    xyz_norm, kpt_scale             = RHD.norm_keypoint(xyz_gt_list[i])
    xyz_gt_canon, inv_rot_mat       = RHD.canonical_transform(xyz_norm)
    rot_mat                         = np.linalg.inv(inv_rot_mat)
    xyz_gt_canon                    = np.squeeze(xyz_gt_canon)

    if hand_side_list[i] == 1:
        xyz_gt_canon[:, 2] = -xyz_gt_canon[:, 2]

    gt_canon_list.append(xyz_gt_canon)
    rot_mat_list.append(rot_mat)
    kpt_scale_list.append(kpt_scale)

In [ ]:
pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_xyz.txt'.format(epoch, data_split))
pred_xyz = np.reshape(np.loadtxt(pred_file), (-1, 21, 3))

pred_uvd = []
for i, xyz in enumerate(pred_xyz):
    pred_uvd.append(RHD.xyz2uvd(xyz, K_list[i]))
pred_uvd = np.asarray(pred_uvd)

pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_canon.txt'.format(epoch, data_split))
pred_canon = np.reshape(np.loadtxt(pred_file), (-1, 21, 3))

pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_rot.txt'.format(epoch, data_split))
pred_rot = np.reshape(np.loadtxt(pred_file), (-1, 3, 3))

In [ ]:
canon_gt_list_compare = np.asarray(gt_canon_list[:len(pred_xyz)])
rot_mat_list_compare = np.asarray(rot_mat_list[:len(pred_xyz)])

xyz_error = RHD.mean_L2_error(pred_canon, canon_gt_list_compare)
print('%s CANON mean_l2_error: ' %data_split, xyz_error) 

xyz_error = RHD.mean_L2_error(pred_rot, rot_mat_list_compare)
print('%s ROT mean_l2_error: ' %data_split, xyz_error) 

In [ ]:
xyz_gt_list_compare = np.asarray(xyz_gt_list[:len(pred_xyz)])

_, xyz_error = RHD.mean_L2_error_vis(xyz_gt_list_compare*1000, pred_xyz*1000, vis_list) # m to mm
print('%s XYZ mean_l2_error: ' %data_split, xyz_error) 

error, uv_error = RHD.mean_L2_error_vis(uv_gt_list[:len(pred_uvd)], pred_uvd[:, :, :], vis_list)
print('%s UV mean_l2_error: ' %data_split, uv_error)
mean, median = RHD.get_median_mean(xyz_gt_list_compare*1000, pred_xyz*1000, vis_list)

print('XYZ calculated per keypoint mean:', mean, 'median:', median)

min_error_idx = np.argmin(error)
max_error_idx = np.argmax(error)
print('Best Pose id:', min_error_idx, 'uv_error:', error[min_error_idx])
print('Worst Pose id:', max_error_idx, 'uv_error:', error[max_error_idx])
for idx in np.argsort(error):
    print(idx, error[idx])

In [ ]:
pck = RHD.percentage_frames_within_error_curve_vis(xyz_gt_list_compare*1000, pred_xyz*1000, vis_list)
print(pck)
thresholds = np.arange(0, 85, 5)
print('AUC:', RHD.calc_auc(pck, thresholds))

In [ ]:
idx = 1       
image = np.asarray(Image.open(os.path.join(DATA_DIR, 'RHD_published_v2', split_set, 
                                             'color', '%.5d.png' % idx)))

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image)
cur_vis = vis_list[idx]
# RHD.visualize_joints_2d(ax, pred_uvd[idx][RHD.REORDER_IDX], c='r')
# RHD.visualize_joints_2d(ax, uv_gt_list[idx][RHD.REORDER_IDX], c='b')
ax.plot(uv_gt_list[idx][cur_vis][:, 0], uv_gt_list[idx][cur_vis][:, 1], 'bo')
ax.plot(pred_uvd[idx][cur_vis][:, 0], pred_uvd[idx][cur_vis][:, 1], 'ro')